In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

## Download Model files from Tensorflow model ZOO
Model files can be downloaded from the Tensorflow Object Detection Model Zoo
https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md

**A sample mobilenet model has been included**

## Create config file from frozen graph
Once you download the model, extract the files and run the tf_text_graph_ssd.py file 
with input as the path to the frozen_graph.pb file and output as desired.
The python file is available at https://github.com/opencv/opencv/blob/master/samples/dnn/tf_text_graph_ssd.py

**A sample config file has been included**

In [ ]:
modelFile = "models/ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb"
configFile = "models/ssd_mobilenet_v2_coco_2018_03_29.pbtxt"
image_dir = "images"
threshold = 0.5
classFile = "coco_class_labels.txt"

In [ ]:
with open(classFile) as fi:
    labels = fi.readline().strip().split()
print(labels)

In [ ]:
# Read the Tensorflow network
net = cv2.dnn.readNetFromTensorflow(modelFile, configFile)

# For ach file in the directory
for filename in os.listdir(image_dir):
    frame = cv2.imread(os.path.join(image_dir, filename))

    # Resize the image to the dimension required for SSD
    frame = cv2.resize(frame, (300, 300))
    rows = frame.shape[0]
    cols = frame.shape[1]

    # Create a blob from the image and pass it to the network
    net.setInput(cv2.dnn.blobFromImage(frame, 1.0/127.5, (300, 300), (127.5, 127.5, 127.5), True))
    
    # Peform Prediction
    out = net.forward()
    
    # For every Detected Object
    for i in range(out.shape[2]):
        # Find the class and confidence 
        classId = int(out[0, 0, i, 1])
        score = float(out[0, 0, i, 2])
        
        # The locations given are normalized. They should be multiplied by the height and width of the input
        x = int(out[0, 0, i, 3] * cols)
        y = int(out[0, 0, i, 4] * rows)
        w = int(out[0, 0, i, 5] * cols - x)
        h = int(out[0, 0, i, 6] * rows - y)
        
        # Check if the detection is of good quality
        if score > threshold:
            cv2.putText(frame, "{}".format(labels[classId]), ( x, y ), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 1, cv2.LINE_AA)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 255), cv2.FONT_HERSHEY_DUPLEX)
    
    # Convert Image to RGB since we are using Matplotlib for displaying image
    mp_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.imshow(mp_img)
    plt.show()